In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tinyllama-final/adapter_model.safetensors
/kaggle/input/tinyllama-final/trainer_state.json
/kaggle/input/tinyllama-final/training_args.bin
/kaggle/input/tinyllama-final/adapter_config.json
/kaggle/input/tinyllama-final/README.md
/kaggle/input/tinyllama-final/tokenizer.json
/kaggle/input/tinyllama-final/tokenizer_config.json
/kaggle/input/tinyllama-final/scaler.pt
/kaggle/input/tinyllama-final/scheduler.pt
/kaggle/input/tinyllama-final/special_tokens_map.json
/kaggle/input/tinyllama-final/optimizer.pt
/kaggle/input/tinyllama-final/rng_state.pth
/kaggle/input/tinyllama-final/tokenizer.model
/kaggle/input/rewardmodel/adapter_model.safetensors
/kaggle/input/rewardmodel/training_args.bin
/kaggle/input/rewardmodel/adapter_config.json
/kaggle/input/rewardmodel/README.md
/kaggle/input/rewardmodel/tokenizer.json
/kaggle/input/rewardmodel/tokenizer_config.json
/kaggle/input/rewardmodel/special_tokens_map.json
/kaggle/input/rewardmodel/tokenizer.model
/kaggle/input/reward-model/adap

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_path = "/kaggle/input/rewardmodel"  # path where your trained reward model is saved
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=3  # ✅ IMPORTANT: Match training setup
).to("cuda")

model.eval()
print("✅ Reward model loaded successfully.")


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Reward model loaded successfully.


# Define Score funtion

In [5]:
def score_response(prompt, response):
    input_text = f"<|user|>: {prompt} <|assistant|>: {response}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=384).to("cuda")

    with torch.no_grad():
        outputs = model(**inputs)
        score = torch.softmax(outputs.logits, dim=1)[0].tolist()  # probability over rank classes

    return {
        "scores": score,         # e.g., [0.1, 0.3, 0.6] — rank 2 most preferred
        "predicted_rank": int(torch.argmax(outputs.logits))  # e.g., 2
    }


# Sample responses

In [6]:
prompt = "I have sore throat and fever for 2 days. What could be the cause?"

# Three generated outputs
response1 = "You may have a viral infection like the common cold or flu. It's best to stay hydrated and rest."
response2 = "Sore throat and fever may indicate strep throat. Please see a doctor for antibiotics."
response3 = "I think it's just stress. No need to worry about it."

print("🔹 Response 1:", score_response(prompt, response1))
print("🔹 Response 2:", score_response(prompt, response2))
print("🔹 Response 3:", score_response(prompt, response3))


🔹 Response 1: {'scores': [0.036672789603471756, 0.36894407868385315, 0.594383180141449], 'predicted_rank': 2}
🔹 Response 2: {'scores': [0.04787353053689003, 0.0522964708507061, 0.8998300433158875], 'predicted_rank': 2}
🔹 Response 3: {'scores': [0.1386435180902481, 0.05790585279464722, 0.8034506440162659], 'predicted_rank': 2}
